In [1]:
# 참고 사이트 
# 참고소스1 : transformers huggingface mlm 사전 학습 예제
#   - https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
#
# 참고소스2  : bert를 가지고 중국어 corpus로 추가학습시킨 예제
#    - https://github.com/zhusleep/pytorch_chinese_lm_pretrain 에 run_language_model_bert.py 
#      https://www.fatalerrors.org/a/further-pre-training-of-chinese-language-model-bert-roberta.html

import transformers
import numpy as np
import torch

import os
from os import sys
sys.path.append('..')
from myutils import seed_everything, TextDatasetForNextSentencePrediction, GPU_info, mlogging, MyTextDataset
from transformers import BertTokenizer, BertModel, BertTokenizerFast, BertConfig, AutoModelWithLMHead, BertForMaskedLM
from transformers import DataCollatorForLanguageModeling

In [2]:
# wand 비활성화 
# => trainer 로 훈련시키면 기본이 wandb 활성화이므로, 비활성화 시킴
os.environ["WANDB_DISABLED"] = "true"

device = GPU_info()
print(device)

#seed 설정
seed_everything(111)

#logging 설정
logger=mlogging(loggername="bertperplexityeval", logfilename="bertperplexityeval")

True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30
cuda:0
logfilepath:bertperplexityeval_2022-09-13.log


In [3]:
# 변수들 설정
#input_corpus = "Korpora/kowikitext/kowikitext_20200920.test"
input_corpus = "../../data11/my_corpus/bong_eval.txt"


input_model_path = "../../data11/model/bert/mbertV2.0"
vocab_file="../../data11/model/bert/mbertV2.0/vocab.txt"
ouput_model_dir = "../../data11/model/bert/mbertV2.0-1"

'''
input_model_path = "model/bert-multilingual-cased"
vocab_file="model/bert-multilingual-cased/vocab/vocab.txt"
ouput_model_dir = "model/bert-multilingual-cased"
'''
'''
input_model_path = "model/bert-multilingual-cased_furter_pt_model_0216"
vocab_file="model/bert-multilingual-cased_furter_pt_model_0216/vocab/vocab.txt"
ouput_model_dir = "model/bert-multilingual-cased_furter_pt_model_0216"
'''
# 토큰활 할때 최대 길이 
token_max_len = 128

# 훈련용 변수
batch_size = 32   # 128로 하면, GPU Out of memory 발생함(=>**따라서 32로 진행)
 # embedding size 최대 몇 token까지 input으로 사용할 것인지 지정(기본:512) 512, 1024, 2048 식으로 지정함, 엄청난 장문을 다룰경우 10124까지
max_position_embeddings = 128 
logging_steps = 1  # 훈련시, 로깅할 step 수 (크면 10000번 정도하고, 작으면 100번정도)
save_steps = 1     # 10000 step마다 모델 저장
save_total_limit = 1 # 마지막 3개 모델 빼고 과거 모델은 삭제(100000번째마다 모델 저장하는데, 마지감 3개 빼고 나머지는 삭제)


In [4]:
# tokenizer 생성
# => BertTokenizer, BertTokenizerFast 둘중 사용하면됨
'''
tokenizer = BertTokenizer(vocab_file=vocab_file, 
                          max_len=token_max_len, 
                          do_lower_case=False)
'''

#tokenizer = BertTokenizerFast(vocab_speical_path)
tokenizer = BertTokenizerFast(
    vocab_file=vocab_file,
    max_len=token_max_len,
    do_lower_case=False,
    )
    
# fast 토크너나이즈인지 확인
print(f'{vocab_file} is_fast:{tokenizer.is_fast}')

# speical 토큰 계수 + vocab 계수 - 이미 vocab에 포함된 speical 토큰 계수(5)
vocab_size = len(tokenizer.all_special_tokens) + tokenizer.vocab_size - 5 + 1
#vocab_size = len(tokenizer.all_special_tokens) + tokenizer.vocab_size - 5
print('special_token_size: {}, tokenizer.vocab_size: {}'.format(len(tokenizer.all_special_tokens), tokenizer.vocab_size))
print('vocab_size: {}'.format(vocab_size))
print('tokenizer_len: {}'.format(len(tokenizer)))

../../data11/model/bert/mbertV2.0/vocab.txt is_fast:True
special_token_size: 5, tokenizer.vocab_size: 152537
vocab_size: 152538
tokenizer_len: 152537


In [5]:
#output_hidden_states = False # 기본은 False=>output 2개 출력됨, True로 지정하면 output이 3개 출력됨
#return_dict = True   #False로 지정하는 경우 일반적인 tuple을 리턴, True인 경우는 transformers.file_utils.ModelOutput(ouput.logisc) 으로 리턴
#model = BertModel.from_pretrained(input_model_path, output_hidden_states = output_hidden_states, return_dict = return_dict)


# AutoModelForMaskedLM, BertForMaskedLM 
# further pre-training 인 경우 (기존 거에 추가적으로 하는 경우)
config = BertConfig.from_pretrained(input_model_path)
model = BertForMaskedLM.from_pretrained(input_model_path, from_tf=bool(".ckpt" in input_model_path), config=config)

# 모델 embedding 사이즈를 tokenizer +1 크기 만큼 재 설정함.
# => 원래 크기를 tokenizer 사이즈 만큼만 설정하면되는데, kobert는 8002로 평가하면, 오류발생하므로 +1 정도 크게 설정함
model.resize_token_embeddings(len(tokenizer) + 1)
model.eval().to(device)

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(152538, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          

In [6]:
model.num_parameters()

203344602

In [7]:
#==================================================================================================
# load_dataset을 이용하여, 훈련/평가 dataset 로딩.
#
# [로컬 데이터 파일 로딩]
# => dataset = load_dataset("text", data_files='로컬.txt')       # text 로컬 파일 로딩
# => dataset = load_dataset("csv", data_files='로컬.csv')        # csv 로컬 파일 로딩
# => dataset = load_dataset("csv", data_files='로컬.tsv', delimiter="\t")  # tsv 로컬 파일 로딩
# => dataset = load_dataset("json", data_files='로컬.json')      # json 로컬 파일 로딩
# => dataset = load_dataset("pandas", data_files='로컬.pkl')     # pickled dataframe 로컬 파일 로딩
#
# [원격 데이터 파일 로딩]
# url = "https://github.com/crux82/squad-it/raw/master/"
# data_files = {
#    "train": url + "SQuAD_it-train.json.gz",
#    "test": url + "SQuAD_it-test.json.gz",
# }
# squad_it_dataset = load_dataset("json", data_files=data_files, field="data")
#
# 출처 : https://wikidocs.net/166816
#==================================================================================================

from datasets import load_dataset

# 평가 말뭉치 로딩
#train_dataset = load_dataset(input_corpus)
eval_dataset = load_dataset("text", data_files=input_corpus, split="train") # text 로컬 파일 로딩

# eval_dataset 출력해봄
print(f"eval_dataset========================================")
print(eval_dataset)

Using custom data configuration default-91bccccf01bee1c6
Reusing dataset text (/MOCOMSYS/.cache/huggingface/datasets/text/default-91bccccf01bee1c6/0.0.0/08f6fb1dd2dab0a18ea441c359e1d63794ea8cb53e7863e6edf8fc5655e47ec4)


eval_dataset========================================
Dataset({
    features: ['text'],
    num_rows: 1500
})


In [8]:
# eval_dataset 출력해봄
print(f"eval_dataset========================================")
print(eval_dataset)
print(eval_dataset['text'][0:3])

eval_dataset========================================
Dataset({
    features: ['text'],
    num_rows: 1500
})
['공공지원 민간임대주택은 주변 시세의 95  이하의 저렴한 임대료 혜택 외에도 품질좋은 주택에서 8년간 안심하고 살 수 있는 장점이 있다 ', '위기아동 발생을 미리 예방할 수 있는 지원활동이나 인프라 구축보다는 피해아동이 발생했을 때 그때그때 대처하는 방식에서 벗어나지 못하고 있다는 것이다 ', '여야가 각종 현안마다 정치적 타협을 모색하는 대신 검찰과 법원을 찾아 상대를 처벌해 달라고 독촉하는  정치의 사법화 가 극심해지고 있다는 지적이다 ']


In [9]:
# tokenizer 처리
def tokenizer_function(examples):
    result =  tokenizer(examples['text'], truncation=True, max_length=token_max_len, return_overflowing_tokens=True)
    
    # 신규 인덱스와 이전 인덱스와의 매핑 추출
    sample_map = result.pop("overflow_to_sample_mapping")
    for key, values in examples.items():
        result[key] = [values[i] for i in sample_map]
    return result


# batched=True 하면 빠른 tokenizer 이용(Rust)
%time eval_dataset_fast = eval_dataset.map(tokenizer_function, batched=True)

'''
%time tokenized_dataset = text_dataset.map(tokenizer_function, batched=False)
print(tokenized_dataset_fast['train']['text'][0:2])
'''

Loading cached processed dataset at /MOCOMSYS/.cache/huggingface/datasets/text/default-91bccccf01bee1c6/0.0.0/08f6fb1dd2dab0a18ea441c359e1d63794ea8cb53e7863e6edf8fc5655e47ec4/cache-d6811f5a8acb686f.arrow


CPU times: user 48.2 ms, sys: 6.36 ms, total: 54.6 ms
Wall time: 52.4 ms


"\n%time tokenized_dataset = text_dataset.map(tokenizer_function, batched=False)\nprint(tokenized_dataset_fast['train']['text'][0:2])\n"

In [10]:
print(f"eval_dataset_fast=======================================")
print(eval_dataset_fast)
print(f'*fast_len:{len(eval_dataset_fast)}, len:{len(eval_dataset)}')  # fast_dataset과 dataset 길이를 비교함
print(eval_dataset_fast[0:2])

eval_dataset_fast=======================================
Dataset({
    features: ['text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1500
})
*fast_len:1500, len:1500
{'text': ['공공지원 민간임대주택은 주변 시세의 95  이하의 저렴한 임대료 혜택 외에도 품질좋은 주택에서 8년간 안심하고 살 수 있는 장점이 있다 ', '위기아동 발생을 미리 예방할 수 있는 지원활동이나 인프라 구축보다는 피해아동이 발생했을 때 그때그때 대처하는 방식에서 벗어나지 못하고 있다는 것이다 '], 'input_ids': [[101, 120673, 120141, 121218, 151323, 125141, 10892, 120118, 136752, 10459, 11978, 121775, 10459, 125634, 11102, 142626, 125037, 9597, 35979, 120363, 119214, 10892, 122174, 11489, 129, 10954, 18784, 132432, 12453, 9408, 9460, 13767, 121850, 10739, 11506, 102], [101, 120839, 134836, 119574, 10622, 121162, 121857, 14843, 9460, 13767, 119577, 120530, 43739, 121103, 119647, 80001, 11018, 120125, 134836, 10739, 119574, 119424, 10622, 9137, 121927, 142642, 122805, 12178, 119640, 11489, 121768, 12508, 9290, 12453, 77324, 24190, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [11]:
# MLM을 위한 DataCollatorForLangunageModeling 호출
from transformers import DataCollatorForLanguageModeling

# input_ids에 대해 MLM 만들기
data_collator = DataCollatorForLanguageModeling(    # [MASK] 를 씌우는 것은 저희가 구현하지 않아도 됩니다! :-)
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

# input_ids MLM 만들고 출력 해봄
mlm_eval_sample = data_collator(eval_dataset_fast['input_ids'][0:2])

print(f"eval_dataset_fast(MLM)=======================================")
print(mlm_eval_sample['input_ids'][0])
print(eval_dataset_fast[0])

eval_dataset_fast(MLM)=======================================
tensor([   101, 120673, 120141, 121218, 151323, 125141,  10892,    103,    103,
         10459,  11978, 121775,  10459, 125634,  11102, 142626, 125037,   9597,
         35979, 120363, 119214,  10892,    103,  11489,    129,  10954,    103,
        132432,  12453,   9408,   9460,  13767, 121850,  10739,  11506,    102,
             0])
{'text': '공공지원 민간임대주택은 주변 시세의 95  이하의 저렴한 임대료 혜택 외에도 품질좋은 주택에서 8년간 안심하고 살 수 있는 장점이 있다 ', 'input_ids': [101, 120673, 120141, 121218, 151323, 125141, 10892, 120118, 136752, 10459, 11978, 121775, 10459, 125634, 11102, 142626, 125037, 9597, 35979, 120363, 119214, 10892, 122174, 11489, 129, 10954, 18784, 132432, 12453, 9408, 9460, 13767, 121850, 10739, 11506, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [12]:
# load_metric 사용을 위해서는 datasets, sklearn 패키지 설치해야함
#!pip install datasets
#!pip install sklearn

# 참고 소스 : https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py

import datasets
from datasets import load_dataset, load_metric

def preprocess_logits_for_metrics1(logits, labels):
    return logits.argmax(dim=-1)

metric = load_metric("accuracy")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # preds have the same shape as the labels, after the argmax(-1) has been calculated
    # by preprocess_logits_for_metrics
    labels = labels.reshape(-1)
    preds = preds.reshape(-1)
    mask = labels != -100
    labels = labels[mask]
    preds = preds[mask]
    return metric.compute(predictions=preds, references=labels)

In [13]:
from transformers import Trainer, TrainingArguments

# cpu 사용이면 'no_cuda = True' 설정함.
no_cuda = False
if device == 'cpu':
    no_cuda = True
print(f'*no_cuda: {no_cuda}')

training_args = TrainingArguments(
    no_cuda = no_cuda,                      # GPU 사용  안함
    output_dir=ouput_model_dir,
    per_device_eval_batch_size=batch_size
    #per_gpu_eval_batch_size=batch_size
)


eval_dataset_fast_input_ids = eval_dataset_fast['input_ids']

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,  #data
    eval_dataset=eval_dataset_fast_input_ids,
    compute_metrics=compute_metrics,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics1
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


*no_cuda: False


In [14]:
###################################################################
# 펄플렉서티(Perplexity, PPL) 로 성능 평가함 
# => PPL은 선택할 수 있는 가능한 경우의 수를 의미하는 분기계수(branching factor)입니다. 
#   PPL은 이 언어 모델이 특정 시점에서 평균적으로 몇 개의 선택지를 가지고 고민하고 있는지를 의미합니다. 
#  가령, 언어 모델에 어떤 테스트 데이터을 주고 측정했더니 PPL이 10이 나왔다고 해봅시다. 
#  그렇다면 해당 언어 모델은 테스트 데이터에 대해서 다음 단어를 예측하는 모든 시점(time step)마다 
#  평균 10개의 단어를 가지고 어떤 것이 정답인지 고민하고 있다고 볼 수 있습니다. 
#  같은 테스트 데이터에 대해서 두 언어 모델의 PPL을 각각 계산 후에 PPL의 값을 비교하면, 
#  두 언어 모델 중 PPL이 더 낮은 언어 모델의 성능이 더 좋다고 볼 수 있습니다.
###################################################################
eval_output = trainer.evaluate()

***** Running Evaluation *****
  Num examples = 1500
  Batch size = 32


In [15]:
import math
import os
# Evaluation
results = {}

perplexity = math.exp(eval_output["eval_loss"])
result = {"perplexity": perplexity}

output_eval_file = os.path.join(ouput_model_dir, "eval_results_lm.txt")
with open(output_eval_file, "w") as writer:
    logger.info("***** Eval results *****")
    for key in sorted(result.keys()):
        logger.info("%s:  %s = %s", input_model_path, key, str(result[key]))
        writer.write("%s = %s\n" % (key, str(result[key])))

results.update(result)


2022-09-13 13:30:22,357 - bertperplexityeval - INFO - ***** Eval results *****
2022-09-13 13:30:22,359 - bertperplexityeval - INFO - ../../data11/model/bert/mbertV2.0:  perplexity = 19.781589884565754


In [16]:
# OUT OF MEMORY 에러면 GPU 사용량 체크
#!nvidia-smi

Tue Sep 13 13:30:22 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A30          Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   29C    P0    44W / 165W |  14071MiB / 24258MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------